# Import data

In [2]:
import pandas as pd 
pd.set_option('display.max_rows', None)
pd.set_option('max_colwidth', None)

df = pd.read_csv('popularity.csv', delimiter=';').sort_values(by='Popularity', ascending=False).head(10)
df.drop("Popularity", axis=1, inplace=True)
df

,Challenge,Predicate,Expression
1962,dkZH6HJNQNLLDX6Aj,inv5,all inf : Influencer | all u: User | inf in u.follows
1979,dkZH6HJNQNLLDX6Aj,inv5,"all u:User, i:Influencer | i in u.follows"
17,dkZH6HJNQNLLDX6Aj,inv1,all x : Photo | some y : User | y->x in posts
2695,dkZH6HJNQNLLDX6Aj,inv7,all u : User | u.suggested in u.follows.follows and u.suggested not in u.follows
2703,dkZH6HJNQNLLDX6Aj,inv7,"all disj u,uu:User | u in uu.follows.follows && u not in uu.follows implies u in uu.suggested"
1978,dkZH6HJNQNLLDX6Aj,inv5,all x : Influencer | all p : User | p in follows.x
20,dkZH6HJNQNLLDX6Aj,inv1,all p : Photo | p in User.posts
2736,dkZH6HJNQNLLDX6Aj,inv7,"all u1,u2 : User | u2 in u1.suggested implies u2 in u1.follows.follows and u2 not in u1.follows"
1515,dkZH6HJNQNLLDX6Aj,inv4,all x:User| some a:Ad| a in x.posts implies x.posts in Ad
2721,dkZH6HJNQNLLDX6Aj,inv7,all u : User | u.suggested = u.follows.follows - u.follows


# Request hints

In [3]:
import requests

def send_http_request(url, body):
    try:
        response = requests.post(url, json=body)
        # Check if the request was successful (status code 200)
        if response.status_code == 200:
            return response.json()
        else:
            print(f"Request failed with status code {response.status_code}")
    except requests.RequestException as e:
        print(f"An error occurred: {e}")
    return None

def gen_body_request(model, challenge, predicate, expression, hintGenType):
    obj = {
        "model": model + " pred " + predicate + " { " + expression + " }",
        "challenge": challenge,
        "predicate": predicate,
        "hintGenType": hintGenType
    }

    return obj

def get_hint(model, url, challenge, predicate, expression, hintGenType = "TED"):
    body = gen_body_request(model, challenge, predicate, expression, hintGenType)
    response = send_http_request(url, body)
    1
    if response is not None:
        return pd.Series([response["hint"], response["nextExpr"], response["targetExpr"]])
    else:
        return pd.Series(["", "", ""])

In [4]:
model = "sig User {follows : set User,sees : set Photo,posts : set Photo,suggested : set User} sig Influencer extends User {} sig Photo {date : one Day} sig Ad extends Photo {} sig Day {}"
url = "http://localhost:8080/hint/higena-hint"

# Hint using Higena with TED
higenaTED = df.copy()
columns = df.apply(lambda row: get_hint(model, url, row["Challenge"], row["Predicate"], row["Expression"]), axis=1)
higenaTED[["hint", "next", "solution"]] = columns

higenaTED

,Challenge,Predicate,Expression,hint,next,solution
1962,dkZH6HJNQNLLDX6Aj,inv5,all inf : Influencer | all u: User | inf in u.follows,Near a solution! Consider adding a difference operator ('-') to remove elements from a set. Think about how you can incorporate this within the unique quantifier ('one') expression.,all x : Influencer | all y : User-x | x in y.follows,all x : Influencer | all y : User-x | x in y.follows
1979,dkZH6HJNQNLLDX6Aj,inv5,"all u:User, i:Influencer | i in u.follows","Keep going! Consider adding a implication operator ('=>') to specify that if the left side is true, then the right side must also be true. Think about how you can incorporate this within the universal quantifier ('all') expression.","all u : User, i : Influencer | i!=u implies i in u.follows","all u : User, i : Influencer | i!=u implies i in u.follows"
17,dkZH6HJNQNLLDX6Aj,inv1,all x : Photo | some y : User | y->x in posts,"One step away from the solution! Instead of using existential quantifier ('some') to specify that some elements in a set satisfy a condition, try using unique quantifier ('one') to specify that there is exactly one element in a set.",all x : Photo | one y : User | y -> x in posts,all x : Photo | one y : User | y -> x in posts
2695,dkZH6HJNQNLLDX6Aj,inv7,all u : User | u.suggested in u.follows.follows and u.suggested not in u.follows,"Keep going! Instead of using conjunction operator ('and') to combine two boolean expressions, try using equal operator ('=') to specify that the left side is equal to the right side.",all u : User | u.suggested = (u.follows.follows - u.follows) - u,all u : User | u.suggested = (u.follows.follows - u.follows) - u
2703,dkZH6HJNQNLLDX6Aj,inv7,"all disj u,uu:User | u in uu.follows.follows && u not in uu.follows implies u in uu.suggested","Keep going! It seems like you have unnecessary elements in your expression. You can try simplifying your expression by deleting the disjoint operator ('disj'). If you want to keep it, try to fix your expression another way and reach a different solution!","all a,b : User | a in b.follows.follows && a not in b.follows => a in b.suggested","all u1,u2:User | (u1!=u2 and u1 in u2.follows.follows and u1 not in u2.follows) <=> (u1 in u2.suggested)"
1978,dkZH6HJNQNLLDX6Aj,inv5,all x : Influencer | all p : User | p in follows.x,Near a solution! Consider adding a difference operator ('-') to remove elements from a set. Think about how you can incorporate this within the unique quantifier ('one') expression.,all x : Influencer | (all y : User-x | y in follows.x),all x : Influencer | (all y : User-x | y in follows.x)
20,dkZH6HJNQNLLDX6Aj,inv1,all p : Photo | p in User.posts,"Keep going! Instead of using inclusion operator ('in') to specify that some element(s) belong to a set, try using unique quantifier ('one') to specify that there is exactly one element in a set.",all x:Photo | one posts.x,all x:Photo | one posts.x
2736,dkZH6HJNQNLLDX6Aj,inv7,"all u1,u2 : User | u2 in u1.suggested implies u2 in u1.follows.follows and u2 not in u1.follows",Keep going! Consider adding a equivalence operator ('<=>') to specify the equivalence of the right and left side of the expression. Think about how you can incorporate this within the universal quantifier ('all') expression.,"all u,s : User | s in u.suggested iff( s in u.follows.follows and s not in u.follows and u!=s)","all u,s : User | s in u.suggested iff( s in u.follows.follows and s not in u.follows and u!=s)"
1515,dkZH6HJNQNLLDX6Aj,inv4,all x:User| some a:Ad| a in x.posts implies x.posts in Ad,"One step away from the solution! Instead of using existential quantifier ('some') to specify that some elements in a set satisfy a condition, try using universal quantifier ('all') to specify that all elements in a set satisfy a condition.",all u:User | all a:Ad | a in u.posts implies u.posts in Ad,all u:User | all a:Ad | a in u.posts implies u.posts in Ad
2721,dkZH6HJNQNLLDX6Aj,inv7,all u : User | u.suggested = u.follows.follows - u.follow

In [5]:
# Hint using Higena with Most popular submissions
higenaPopular = df.copy()
columns = df.apply(lambda row: get_hint(model, url, row["Challenge"], row["Predicate"], row["Expression"], hintGenType="NODE_POISSON"), axis=1)
higenaPopular[["hint", "next", "solution"]] = columns

higenaPopular

,Challenge,Predicate,Expression,hint,next,solution
1962,dkZH6HJNQNLLDX6Aj,inv5,all inf : Influencer | all u: User | inf in u.follows,"Keep going! Consider adding a implication operator ('=>') to specify that if the left side is true, then the right side must also be true. Think about how you can incorporate this within the universal quantifier ('all') expression.","all x:Influencer, y:User | x!=y implies x in y.follows","all x:Influencer, y:User | x!=y implies x in y.follows"
1979,dkZH6HJNQNLLDX6Aj,inv5,"all u:User, i:Influencer | i in u.follows","Keep going! Instead of using signature of type User, try using signature of type Influencer to help satisfy the required property.","all x:Influencer, y:User | x!=y implies x in y.follows","all x:Influencer, y:User | x!=y implies x in y.follows"
17,dkZH6HJNQNLLDX6Aj,inv1,all x : Photo | some y : User | y->x in posts,"Keep going! Instead of using existential quantifier ('some') to specify that some elements in a set satisfy a condition, try using unique quantifier ('one') to specify that there is exactly one element in a set.",all x:Photo | one posts.x,all x:Photo | one posts.x
2695,dkZH6HJNQNLLDX6Aj,inv7,all u : User | u.suggested in u.follows.follows and u.suggested not in u.follows,"Keep going! Instead of using conjunction operator ('and') to combine two boolean expressions, try using equal operator ('=') to specify that the left side is equal to the right side.",all u : User | u.suggested = (u.follows.follows - u.follows) - u,all u : User | u.suggested = (u.follows.follows - u.follows) - u
2703,dkZH6HJNQNLLDX6Aj,inv7,"all disj u,uu:User | u in uu.follows.follows && u not in uu.follows implies u in uu.suggested","Keep going! Instead of using implication operator ('=>') to specify that if the left side is true, then the right side must also be true, try using equal operator ('=') to specify that the left side is equal to the right side.",all u : User | u.suggested = (u.follows.follows - u.follows) - u,all u : User | u.suggested = (u.follows.follows - u.follows) - u
1978,dkZH6HJNQNLLDX6Aj,inv5,all x : Influencer | all p : User | p in follows.x,"Keep going! Consider adding a implication operator ('=>') to specify that if the left side is true, then the right side must also be true. Think about how you can incorporate this within the universal quantifier ('all') expression.","all x:Influencer, y:User | x!=y implies x in y.follows","all x:Influencer, y:User | x!=y implies x in y.follows"
20,dkZH6HJNQNLLDX6Aj,inv1,all p : Photo | p in User.posts,"Keep going! Instead of using inclusion operator ('in') to specify that some element(s) belong to a set, try using unique quantifier ('one') to specify that there is exactly one element in a set.",all x:Photo | one posts.x,all x:Photo | one posts.x
2736,dkZH6HJNQNLLDX6Aj,inv7,"all u1,u2 : User | u2 in u1.suggested implies u2 in u1.follows.follows and u2 not in u1.follows","Keep going! Instead of using implication operator ('=>') to specify that if the left side is true, then the right side must also be true, try using equal operator ('=') to specify that the left side is equal to the right side.",all u : User | u.suggested = (u.follows.follows - u.follows) - u,all u : User | u.suggested = (u.follows.follows - u.follows) - u
1515,dkZH6HJNQNLLDX6Aj,inv4,all x:User| some a:Ad| a in x.posts implies x.posts in Ad,"One step away from the solution! Instead of using existential quantifier ('some') to specify that some elements in a set satisfy a condition, try using universal quantifier ('all') to specify that all elements in a set satisfy a condition.",all u:User | all a:Ad | a in u.posts implies u.posts in Ad,all u:User | all a:Ad | a in u.posts implies u.posts in Ad
2721,dkZH6HJNQNLLDX6Aj,inv7,all u : User | u.suggested = u.follows.follows - u.follows,Near a solution! Consider adding a difference operator ('-') to remove elements from a set. Think about how you can incorporate this within the equal operator ('=') expression.,all u : User | u.suggested

In [6]:
# Hint using Spec Assistant
url = "http://localhost:8080/hint/spec-hint"
spec = df.copy()
columns = df.apply(lambda row: get_hint(model, url, row["Challenge"], row["Predicate"], row["Expression"]), axis=1)
spec[["hint", "next", "solution"]] = columns
spec

,Challenge,Predicate,Expression,hint,next,solution
1962,dkZH6HJNQNLLDX6Aj,inv5,all inf : Influencer | all u: User | inf in u.follows,Near a solution! Consider adding a difference operator ('-') to remove elements from a set. Think about how you can incorporate this within the one of expression.,(all ref0:(one Influencer)|(all ref1:(one (User - ref0))|(ref0 in (ref1 . (User <: follows))))),
1979,dkZH6HJNQNLLDX6Aj,inv5,"all u:User, i:Influencer | i in u.follows","Keep going! Instead of using signature of type User, try using signature of type Influencer to help satisfy the required property.",(all ref0:(one Influencer)|(all ref1:(one (User - ref0))|(ref0 in (ref1 . (User <: follows))))),
17,dkZH6HJNQNLLDX6Aj,inv1,all x : Photo | some y : User | y->x in posts,"One step away from the solution! Instead of using existential quantifier ('some') to specify that some elements in a set satisfy a condition, try using unique quantifier ('one') to specify that there is exactly one element in a set.",(all ref0:(one Photo)|(one ref1:(one User)|((ref1 -> ref0) in (User <: posts)))),
2695,dkZH6HJNQNLLDX6Aj,inv7,all u : User | u.suggested in u.follows.follows and u.suggested not in u.follows,"One step away from the solution! Instead of using conjunction operator ('and') to combine two boolean expressions, try using disjunction operator ('or') to combine two boolean expressions.",(all ref0:(one User)|(((ref0 . (User <: suggested)) !in (ref0 . (User <: follows))) || ((ref0 . (User <: suggested)) in ((ref0 . (User <: follows)) . (User <: follows))))),
2703,dkZH6HJNQNLLDX6Aj,inv7,"all disj u,uu:User | u in uu.follows.follows && u not in uu.follows implies u in uu.suggested",Keep going! It seems like the dot join operator ('.') is not in the right place. Try moving it to the inside of the inclusion operator ('in') expression. Try moving it so that you correctly ensure the required property.,"(all ref0:(one User)|(all ref1:(one User)|(disj[ref0,ref1] && (((ref0 !in (ref1 . (User <: follows))) && (ref0 in (ref1 . (^ (User <: follows))))) => (ref0 in (ref1 . (User <: suggested)))))))",
1978,dkZH6HJNQNLLDX6Aj,inv5,all x : Influencer | all p : User | p in follows.x,Near a solution! Consider adding a difference operator ('-') to remove elements from a set. Think about how you can incorporate this within the one of expression.,(all ref0:(one Influencer)|(all ref1:(one (User - ref0))|(ref1 in ((User <: follows) . ref0)))),
20,dkZH6HJNQNLLDX6Aj,inv1,all p : Photo | p in User.posts,Keep going! Consider adding a unique quantifier ('one') to specify that there is exactly one element in a set. Think about how you can incorporate this within the universal quantifier ('all') expression.,(all ref0:(one Photo)|(one ref1:(one User)|(ref0 in (ref1 . (User <: posts))))),
2736,dkZH6HJNQNLLDX6Aj,inv7,"all u1,u2 : User | u2 in u1.suggested implies u2 in u1.follows.follows and u2 not in u1.follows",Keep going! Consider adding a conjunction operator ('and') to combine two boolean expressions. Think about how you can incorporate this within the implication operator ('=>') expression.,(all ref0:(one User)|(all ref1:(one User)|(((ref0 != ref1) && (ref1 in (ref0 . (User <: suggested)))) => ((ref1 !in (ref0 . (User <: follows))) && (ref1 in ((ref0 . (User <: follows)) . (User <: follows))))))),
1515,dkZH6HJNQNLLDX6Aj,inv4,all x:User| some a:Ad| a in x.posts implies x.posts in Ad,"One step away from the solution! Instead of using existential quantifier ('some') to specify that some elements in a set satisfy a condition, try using universal quantifier ('all') to specify that all elements in a set satisfy a condition.",(all ref0:(one User)|(all ref1:(one Ad)|((ref1 in (ref0 . (User <: posts))) => ((ref0 . (User <: posts)) in Ad)))),
2721,dkZH6HJNQNLLDX6Aj,inv7,all u : User | u.suggested = u.follows.follows - u.follows,Near a solution! Consider adding a difference operator ('-') to remove elements from a set. Think about how you can incorporate this within the equal operator ('=') e

# Check for equivalent hints

In [10]:
df["ted"] = higenaTED["hint"]
df["popular"] = higenaPopular["hint"]
df["spec"] = spec["hint"]

print("TED = popular: " + (df["ted"]== df["popular"]).sum().astype(str))
print("TED = spec: " + (df["ted"]== df["spec"]).sum().astype(str))
print("Popular = spec: " + (df["popular"]== df["spec"]).sum().astype(str))
print("All equal: " + ((df["ted"] == df["popular"]) & (df["ted"]== df["spec"])).sum().astype(str))

df["Predicate"].value_counts()

TED = popular: 4
TED = spec: 3
Popular = spec: 3
All equal: 2


inv7    4
inv5    3
inv1    2
inv4    1
Name: Predicate, dtype: int64

# Export

In [8]:
# Export hints csv
df.to_csv("hints.csv", index=False, sep=";")
